<a href="https://colab.research.google.com/github/Yiming-xie-2021/aiperson_utility/blob/main/easydataset%E8%87%AA%E5%AE%9A%E4%B9%89%E5%AF%BC%E5%87%BA%E6%95%B0%E6%8D%AE%E5%A4%84%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
"""
该脚本用于将原始 JSONL 数据转换为 ShareGPT 格式。

主要功能：
1. 从输入 JSONL 文件逐行读取数据，每行包含 instruction（人类指令）、chunk（背景资料）、output（AI回答）。
2. 将 instruction 和 chunk 合并，作为 ShareGPT 格式中的 user 字段，并在 chunk 前加上【背景资料】标注。
3. 从 output 字段中提取“思考链”和“答案正文”，分别放入 assistant 的 think 和 answer 字段。
4. 最终输出为新的 JSONL 文件，每行是一个符合 ShareGPT 格式的对话对象。

核心函数：
- parse_output(output_text): 使用正则表达式提取 output 中的思考链和答案正文。
- convert_to_sharegpt(input_file, output_file): 将输入 JSONL 文件转换为 ShareGPT 格式并写入输出文件。

使用方法：
在命令行运行该脚本，指定输入文件和输出文件路径，例如：
    python script.py
会将 input.jsonl 转换为 sharegpt_output.jsonl。
"""


from os import system
import json
import re

SYSTEM_PROMPT="""你是一名地产地图直播领域的专家，长期服务于用户购房决策，擅长根据给定的【背景资料】回答用户提出的问题：
- 从用户的问题和背景资料识别购房者真实意图与隐性焦虑
- 提炼这类购房人群的共性问题
- 根据背景资料用专业但不说教的方式给出能改变认知层级的答案
你的目标不是简单“答题”，而是帮助一类购房用户意识到认知盲区、规避决策风险，并引导其进一步咨询。"""

def parse_output(output_text):
    """
    从 output 字段中提取思考链和答案正文
    """
    # 匹配思考链部分
    think_match = re.search(r'### 思考链(.*?)(?=### 答案正文)', output_text, re.S)
    answer_match = re.search(r'### 答案正文(.*)', output_text, re.S)

    think = think_match.group(1).strip() if think_match else ""
    answer = answer_match.group(1).strip() if answer_match else output_text.strip()

    return think, answer

def convert_to_sharegpt(input_file, output_file):
    """
    将原始 JSONL 转换为 ShareGPT 格式 JSONL

    """
    with open(input_file, "r", encoding="utf-8") as f_in, open(output_file, "w", encoding="utf-8") as f_out:
        for line in f_in:
            if not line.strip():
                continue
            obj = json.loads(line)

            instruction = obj.get("instruction", "")
            chunk = obj.get("chunk", "")
            output_text = obj.get("output", "")

            # 合并 user 内容
            user_content = f"{instruction}\n\n【背景资料】{chunk}"

            # 提取 assistant 的 think 和 answer
            think, answer = parse_output(output_text)

            sharegpt_obj = {
                "messages": [
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": user_content},
                    {"role": "assistant", "think": think, "answer": answer}
                ]
            }

            f_out.write(json.dumps(sharegpt_obj, ensure_ascii=False) + "\n")


convert_to_sharegpt("/content/datasets-WBKsXj0I5CgG-custom-2025-12-19.jsonl", "sharegpt_output.jsonl")
